In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline

In [145]:
training_labels = pd.read_csv("set_entrenamiento.csv", low_memory=False)
labels_predict = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)
data = pd.read_csv("data_set.csv", low_memory = False)

In [146]:
training_labels = training_labels[['person','label']]
training_labels['label'].value_counts()

1    980
0    980
Name: label, dtype: int64

In [147]:
#data = data[['person','days_since_last_session','checkout','promedio dias','sessions','viewed/checkout gral',
#             'total_views', 'viewed product', 'promedio hora','retornos']]
data.head()

,person,Direct,Email,Organic,Paid,Referral,Social,ad campaign hit,brand listing,checkout,...,same_product_max_views,viewed/checkouted min,viewed/checkouted max,viewed/checkout gral,Computer,Smartphone,Tablet,country_Computer,country_Smartphone,country_Tablet
0,0008ed71,2.0,0.0,0.0,0.0,0.0,0.0,0,0,3,...,0.0,-1.0,-1.0,0.00,2.0,0.0,0.0,1.0,0.0,0.0
1,00091926,1.0,0.0,25.0,0.0,0.0,0.0,15,25,2,...,33.0,1.0,0.0,82.00,34.0,0.0,0.0,1.0,0.0,0.0
2,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,1,5,0,...,1.0,0.0,0.0,-1.00,0.0,1.0,0.0,0.0,1.0,0.0
3,000ba417,0.0,0.0,6.0,0.0,0.0,0.0,1,24,6,...,10.0,1.0,0.0,20.25,6.0,0.0,0.0,1.0,0.0,0.0
4,000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,...,3.0,1.0,1.0,1.00,0.0,1.0,0.0,0.0,1.0,0.0


In [148]:
features = pd.merge(training_labels, data, on='person', how='inner')
features = features.fillna(0)
features['label'].value_counts()

1    980
0    980
Name: label, dtype: int64

In [149]:
labels = features['label']
features = features.drop(['label','person'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25)
print("Train: ",len(x_train),"Test: ",len(x_test))

Train:  1470 Test:  490


In [150]:
def auc2(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict(train)),
                            metrics.roc_auc_score(y_test,m.predict(test)))

In [123]:
lg = lgb.LGBMClassifier(silent=False)
param_dist = {"max_depth": [25,50, 75],
              "learning_rate" : [0.01,0.05,0.1],
              "num_leaves": [300,900,1200],
              "n_estimators": [200]
             }
grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 3, scoring="roc_auc", verbose=5)
grid_search.fit(x_train,y_train)
grid_search.best_estimator_

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] learning_rate=0.01, max_depth=25, n_estimators=200, num_leaves=300 
[CV] learning_rate=0.01, max_depth=25, n_estimators=200, num_leaves=300 
[CV] learning_rate=0.01, max_depth=25, n_estimators=200, num_leaves=300 
[CV] learning_rate=0.01, max_depth=25, n_estimators=200, num_leaves=900 
[CV] learning_rate=0.01, max_depth=25, n_estimators=200, num_leaves=900 
[CV] learning_rate=0.01, max_depth=25, n_estimators=200, num_leaves=900 
[CV] learning_rate=0.01, max_depth=25, n_estimators=200, num_leaves=1200 
[CV] learning_rate=0.01, max_depth=25, n_estimators=200, num_leaves=1200 
[CV]  learning_rate=0.01, max_depth=25, n_estimators=200, num_leaves=900, score=0.8607890177262428, total=  19.2s
[CV] learning_rate=0.01, max_depth=25, n_estimators=200, num_leaves=1200 
[CV]  learning_rate=0.01, max_depth=25, n_estimators=200, num_leaves=900, score=0.8401972544315608, total=  19.6s
[CV] learning_rate=0.01, max_depth=50, n_estimators

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   19.8s


[CV]  learning_rate=0.01, max_depth=25, n_estimators=200, num_leaves=300, score=0.8401972544315608, total=  21.2s
[CV] learning_rate=0.01, max_depth=50, n_estimators=200, num_leaves=300 
[CV]  learning_rate=0.01, max_depth=25, n_estimators=200, num_leaves=900, score=0.8149323603891777, total=  21.3s
[CV] learning_rate=0.01, max_depth=50, n_estimators=200, num_leaves=300 
[CV]  learning_rate=0.01, max_depth=25, n_estimators=200, num_leaves=300, score=0.8149323603891777, total=  23.1s
[CV] learning_rate=0.01, max_depth=50, n_estimators=200, num_leaves=900 
[CV]  learning_rate=0.01, max_depth=25, n_estimators=200, num_leaves=300, score=0.8607890177262428, total=  23.7s
[CV] learning_rate=0.01, max_depth=50, n_estimators=200, num_leaves=900 
[CV]  learning_rate=0.01, max_depth=25, n_estimators=200, num_leaves=1200, score=0.8149323603891777, total=  34.8s
[CV] learning_rate=0.01, max_depth=50, n_estimators=200, num_leaves=900 
[CV]  learning_rate=0.01, max_depth=25, n_estimators=200, num_le

[CV]  learning_rate=0.05, max_depth=75, n_estimators=200, num_leaves=900, score=0.8204218312674929, total=  29.5s
[CV] learning_rate=0.1, max_depth=25, n_estimators=200, num_leaves=300 
[CV]  learning_rate=0.05, max_depth=75, n_estimators=200, num_leaves=900, score=0.8371651339464214, total=  25.6s
[CV] learning_rate=0.1, max_depth=25, n_estimators=200, num_leaves=300 
[CV]  learning_rate=0.05, max_depth=75, n_estimators=200, num_leaves=300, score=0.8371651339464214, total=  40.8s
[CV] learning_rate=0.1, max_depth=25, n_estimators=200, num_leaves=300 
[CV]  learning_rate=0.05, max_depth=75, n_estimators=200, num_leaves=300, score=0.8204218312674929, total=  46.1s
[CV] learning_rate=0.1, max_depth=25, n_estimators=200, num_leaves=900 
[CV]  learning_rate=0.05, max_depth=50, n_estimators=200, num_leaves=1200, score=0.8371651339464214, total=  57.7s
[CV] learning_rate=0.1, max_depth=25, n_estimators=200, num_leaves=900 
[CV]  learning_rate=0.05, max_depth=75, n_estimators=200, num_leaves=

[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  3.8min


[CV]  learning_rate=0.1, max_depth=25, n_estimators=200, num_leaves=900, score=0.8325836332133812, total=  23.3s
[CV] learning_rate=0.1, max_depth=50, n_estimators=200, num_leaves=300 
[CV]  learning_rate=0.1, max_depth=25, n_estimators=200, num_leaves=900, score=0.8536252165800348, total=  38.8s
[CV] learning_rate=0.1, max_depth=50, n_estimators=200, num_leaves=300 
[CV]  learning_rate=0.05, max_depth=75, n_estimators=200, num_leaves=1200, score=0.8371651339464214, total=  55.3s
[CV] learning_rate=0.1, max_depth=50, n_estimators=200, num_leaves=900 
[CV]  learning_rate=0.1, max_depth=25, n_estimators=200, num_leaves=900, score=0.8220545115287218, total=  38.6s
[CV] learning_rate=0.1, max_depth=50, n_estimators=200, num_leaves=900 
[CV]  learning_rate=0.1, max_depth=25, n_estimators=200, num_leaves=1200, score=0.8536252165800348, total=  30.5s
[CV] learning_rate=0.1, max_depth=50, n_estimators=200, num_leaves=900 
[CV]  learning_rate=0.1, max_depth=50, n_estimators=200, num_leaves=300,

[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:  5.2min finished


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.05, max_depth=25,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=200, n_jobs=-1, num_leaves=300, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [151]:
d_train = lgb.Dataset(x_train, label=y_train)
params = {"max_depth": 25, "learning_rate" : 0.05, "num_leaves": 300,  "n_estimators": 200}

In [152]:
model2 = lgb.train(params, d_train)
auc2(model2, x_train, x_test)

/home/camila/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:113: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


(0.9999944426434493, 0.8490566037735848)

In [153]:
set_test = pd.merge(data,labels_predict,on = "person", how = "right")
personas = set_test["person"]
set_test = set_test.drop(columns=['person'])

In [154]:
preds_kaggle = model2.predict(set_test)

In [155]:
#convert into binary values
for i in range(0,len(preds_kaggle)):
    if preds_kaggle[i]>=.5:       # setting threshold to .5
       preds_kaggle[i]=1
    else:  
       preds_kaggle[i]=0

In [156]:
preds_kaggle  = preds_kaggle.tolist()
df_kaggel = pd.DataFrame(preds_kaggle)
df_kaggel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19415 entries, 0 to 19414
Data columns (total 1 columns):
0    19415 non-null float64
dtypes: float64(1)
memory usage: 151.8 KB


In [157]:
df_kaggel["person"] = personas
df_kaggel.rename(columns = {0: 'label'},inplace = True) 
df_kaggel = df_kaggel[['person','label']]

In [158]:
df_kaggel["label"].value_counts()

0.0    13669
1.0     5746
Name: label, dtype: int64

In [159]:
df_kaggel[("label")] = df_kaggel[("label")].astype(int)
df_kaggel.head()

,person,label
0,00091926,1
1,00091a7a,1
2,000ba417,0
3,000e4d9e,1
4,000e619d,0


In [160]:
df_kaggel.to_csv('Resultados_light_GBM.csv',index=False)